# Crunching CMIP5 data

In this notebook we give a very brief example of how we can crunch CMIP5 data using NetCDFSCM. This is highly experimental and this notebook is pretty stupid given that it's really a command-line tool.

In [1]:
# NBVAL_IGNORE_OUTPUT
!python ../scripts/crunch_to_scm.py -h

usage: crunch-to-netcdf-scm [-h] [--var-to-crunch VAR_TO_CRUNCH]
                            [--land-mask-threshold LAND_MASK_THRESHOLD] [-f]
                            input output

Crunch netCDF files to NetCDF SCM csv files

positional arguments:
  input                 Root folder of the data to crunch.
  output                Root folder in which to save the output data

optional arguments:
  -h, --help            show this help message and exit
  --var-to-crunch VAR_TO_CRUNCH
                        Variable to crunch
  --land-mask-threshold LAND_MASK_THRESHOLD
                        Minimum fraction of a box which must be land for it to
                        be counted as a land box
  -f, --force           Overwrite any existing files


In [2]:
# NBVAL_IGNORE_OUTPUT
!python ../scripts/crunch_to_scm.py "../tests/test-data/marble-cmip5" "../output-examples/crunched-files" --land-mask-threshold 50 --var-to-crunch "tas"


Crunching:
../tests/test-data/marble-cmip5

to
../output-examples/crunched-files/netcdf-scm-crunched

Making output directory: ../output-examples/crunched-files/netcdf-scm-crunched

Visiting directory 42 of 42. Current directory :: ../tests/test-data/marble-cmip5/cmip5/rcp85/Amon/tas/NorESM1-ME/r1i1p1                                                                                                                                                                                                                                                                                                                                          
Files crunched with NetCDF SCM
timestamp: 20181114 180739

NetCDF SCM version: 0.5.1+10.g0f97c69
input: ../tests/test-data/marble-cmip5
output: ../output-examples/crunched-files/netcdf-scm-crunched
var-to-crunch: tas
land-mask-threshold: 50.0
force: False


Failures


Warnings
Tried to add areacella cube, failed as shown:
Traceback (most recent call last):
  File 